In [1]:
from IPython.display import Image, display

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.config.system_parameters import TriageSettings

from react_agent.src.util.tools_fabric import ToolsFabric

In [2]:
USE_MCP = False
DEBUG_MODE = False
USE_TRIAGE = False

In [3]:
mapping_question = "Explain how 'Payment Terms' is mapped. Start with 'map_invoice1'."
extensibility_question = "As a Public Cloud customer in Spain, can I extend an existing eDocument customer invoice Process?"
peppol_support = "We as Syntax are currently implementing SAP DRC and are following the official SAP help. We are currently at the step “Integration of Peppol Exchange with SAP S/4HANA Cloud” and there at the creation of an S/4HANA Public Cloud communication user. A client certificate should be uploaded for the communication user to be created, as Basic Auth (via user + password) is not supported. The problem: We do not have this certificate. Where can I get this certificate? What format must it be in? We are stuck at this step because we are missing the certificate mentioned and it is also not clear to us what kind of certificate is meant here and where to get it."
france_us = "As a localization developer, I want to enhance the relevancy check in the France eDocument country class with the new source type so that settlement management documents become relevant for the eDocument process and eDocuments are created for the source documents. How can I Achieve this?"

QUERY = france_us

In [4]:
if USE_TRIAGE:
    triage_agent = Triage()

    query_categorization = triage_agent.triage_user_message(user_message=QUERY)
    if DEBUG_MODE:
        print(f"query categorization: {query_categorization}")

In [5]:
if USE_TRIAGE:
    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=query_categorization["category"],
    )
else:
    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=TriageSettings.Categories.ALL,
    )

In [6]:
if USE_MCP:
    async with MultiServerMCPClient(tools) as client:
        agent = ReActAgent(tool_list=client.get_tools())
        if DEBUG_MODE:
            print(agent.create_sys_prompt())
            display(Image(agent.get_agent_graph()))

        await agent.arun_agent_with_input(user_message=QUERY, debug=DEBUG_MODE)
else:
    agent = ReActAgent(tool_list=tools)
    if DEBUG_MODE:
        print(agent.create_sys_prompt())
        display(Image(agent.get_agent_graph()))

    agent.run_agent_with_input(user_message=QUERY, debug=DEBUG_MODE)

In [7]:
run_data = agent.get_execution_data()

run_data.pretty_print()

Agent Run Summary
Model Used:
gpt-4.1

Execution Time: 
47.148 seconds

Token Consumption Statistics
  Input Tokens: 16606
  Output Tokens: 1665
  Total Tokens: 18271

LLM call count: 5

Tools Used:
  Tool #1:
    Name: abap_method_codebase_search
    Arguments:
      query: France eDocument relevancy check
  Tool #2:
    Name: sap_documentation_summary
    Arguments:
      query: settlement management source type relevancy France eDocument
  Tool #3:
    Name: sap_help_lookup
    Arguments:
      query: settlement management eDocument France


In [8]:
print(run_data.final_output)

To enhance the relevancy check in the France eDocument country class so that settlement management documents become relevant for the eDocument process, follow these steps:

**Technical Steps:**
1. **Enhance the ABAP Logic:**
   - In class `cl_edocument_fr_inv_ci` (or its parent), update the `is_relevant` method to include the new source type for settlement management documents. For example:
     ```abap
     IF mo_source->mv_source_type = cl_edoc_source_stlmt_invoice=>gc_src_stlmt_invoice.
       rv_relevant = abap_true.
       RETURN.
     ENDIF.
     ```
   - Ensure this check is added alongside existing source type checks.

2. **Configuration Steps:**
   - Go to the configuration activity: **Assign eDocument Type to Settlement Document Type**.
   - For France, assign the relevant eDocument type (e.g., `FR_INV` for invoices, `FR_CRE` for credit notes) to your settlement management document types.
   - Save your entries.

**Example Table Entry:**
| Settlement Document Type | eDocument